<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/362_EFIA_SentimentAnalysis_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Sentiment Analysis

### Measuring Emotional Signal Without Losing Control

Sentiment matters — but only when it’s handled responsibly.

This code block adds **measured emotional context** to employee feedback while avoiding the risks of opaque or over-interpreted AI sentiment systems. It is deliberately **rule-based for the MVP**, ensuring that sentiment scoring remains explainable, adjustable, and aligned with real business judgment.

---

## Why Sentiment Is Treated as a Signal — Not a Decision

In this agent, sentiment does **not** drive conclusions on its own.

Instead, it:

* Adds context to recurring issues
* Helps distinguish mild friction from high-pressure pain points
* Supports prioritization decisions already grounded in frequency and themes

This prevents a common failure mode:

> Letting emotional language outweigh operational reality.

---

## How Sentiment Is Analyzed (At a High Level)

### Keyword-Based Detection

Each piece of feedback is scanned for:

* Negative indicators (e.g., *frustrated*, *impossible*, *overwhelmed*)
* Positive indicators (e.g., *helpful*, *improve*, *better*)

The system compares how often each type appears and assigns:

* A sentiment label (positive, neutral, negative)
* An intensity score (how strongly that sentiment is expressed)
* A normalized sentiment score for aggregation

There is no hidden interpretation — every result can be traced back to visible rules.

---

## Intensity: Why It Matters

Not all negative feedback is equal.

This system differentiates between:

* Mild inconvenience
* Sustained frustration
* High-pressure, burnout-adjacent signals

Intensity increases when:

* Multiple sentiment keywords appear
* Strong negative indicators are detected

This allows leadership to see **where pressure is highest**, not just where complaints exist.

---

## Sentiment Across the Organization

### `analyze_all_feedback_sentiment`

Rather than treating sentiment in isolation, the agent:

* Scores every feedback entry
* Preserves sentiment alongside the original submission
* Keeps results auditable at the individual level

This supports later aggregation and review.

---

### `calculate_sentiment_summary`

Once sentiment is measured, it is summarized into leadership-friendly views:

* Overall organizational sentiment
* Positive, neutral, and negative counts
* Average intensity across all feedback
* Sentiment broken down by:

  * Department
  * Category (Issues vs Ideas)

This gives leaders immediate answers to questions like:

* “Where is pressure concentrated?”
* “Which teams are expressing the most strain?”
* “Are ideas generally optimistic or cautious?”

---

## Configurable by Design

Just like themes, sentiment logic is **not fixed**.

HR and leadership can:

* Adjust sentiment keywords as language evolves
* Tune intensity scaling
* Add or remove strong indicators based on organizational norms

This keeps sentiment aligned with **how employees actually communicate**, not how a model assumes they do.

---

## A Clear Upgrade Path (Without Risk)

This approach also sets the stage for future enhancements:

### Today (MVP)

* Rule-based
* Transparent
* Easy to audit
* Low risk

### Tomorrow

* LLM-assisted sentiment nuance
* Context-aware tone detection
* Department-specific language patterns

Even then, rule-based sentiment can remain the baseline — with AI acting as an enhancer, not a replacement.

---

## Why Leaders Can Trust This

From a business perspective, this design:

* Avoids overreaction to emotional language
* Prevents black-box sentiment scoring
* Keeps interpretation grounded in facts
* Makes escalation criteria visible and adjustable

Sentiment informs decisions — it doesn’t hijack them.

---

## Architectural Takeaway

This sentiment layer adds **emotional context without sacrificing control**.

By measuring sentiment explicitly and transparently, the agent helps leaders understand not just *what* employees are saying — but *how strongly they feel about it* — while preserving accountability and trust.



# Sentiment Analysis Utilities for Employee Feedback Intelligence Agent

In [ ]:
"""Sentiment Analysis Utilities for Employee Feedback Intelligence Agent

Rule-based sentiment analysis (MVP). Future: Can enhance with LLM for nuanced analysis.
"""

from typing import List, Dict, Any
from collections import Counter, defaultdict


def analyze_sentiment(
    feedback_text: str,
    sentiment_keywords: Dict[str, List[str]]
) -> Dict[str, Any]:
    """
    Analyze sentiment of a single feedback entry using keyword matching.

    Args:
        feedback_text: The feedback text to analyze
        sentiment_keywords: Dictionary mapping sentiment to keyword lists

    Returns:
        Dictionary with sentiment, intensity, and score
    """
    text_lower = feedback_text.lower()

    # Count keyword matches
    negative_count = sum(1 for kw in sentiment_keywords.get("negative", []) if kw in text_lower)
    positive_count = sum(1 for kw in sentiment_keywords.get("positive", []) if kw in text_lower)

    # Determine sentiment
    if negative_count > positive_count:
        sentiment = "negative"
        intensity = min(1.0, 0.3 + (negative_count * 0.15))  # Base 0.3, +0.15 per keyword
        sentiment_score = -intensity  # -1.0 to 0.0
    elif positive_count > negative_count:
        sentiment = "positive"
        intensity = min(1.0, 0.3 + (positive_count * 0.15))
        sentiment_score = intensity  # 0.0 to 1.0
    else:
        sentiment = "neutral"
        intensity = 0.5
        sentiment_score = 0.0

    # Adjust for strong negative indicators
    strong_negative_indicators = ["impossible", "frustrated", "overwhelmed", "broken", "terrible"]
    if any(indicator in text_lower for indicator in strong_negative_indicators):
        if sentiment == "negative":
            intensity = min(1.0, intensity + 0.2)
            sentiment_score = -intensity

    return {
        "sentiment": sentiment,
        "intensity": round(intensity, 2),
        "sentiment_score": round(sentiment_score, 2)
    }


def analyze_all_feedback_sentiment(
    feedback: List[Dict[str, Any]],
    sentiment_keywords: Dict[str, List[str]]
) -> List[Dict[str, Any]]:
    """
    Analyze sentiment for all feedback entries.

    Args:
        feedback: List of feedback entries
        sentiment_keywords: Dictionary mapping sentiment to keyword lists

    Returns:
        List of sentiment analysis results (one per feedback entry)
    """
    sentiment_results = []

    for entry in feedback:
        submission_id = entry.get("submission_id")
        feedback_text = entry.get("free_text_feedback", "")

        sentiment_analysis = analyze_sentiment(feedback_text, sentiment_keywords)

        sentiment_results.append({
            "submission_id": submission_id,
            **sentiment_analysis
        })

    return sentiment_results


def calculate_sentiment_summary(
    feedback: List[Dict[str, Any]],
    sentiment_analysis: List[Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Calculate aggregated sentiment summary statistics.

    Args:
        feedback: List of feedback entries
        sentiment_analysis: List of sentiment analysis results

    Returns:
        Summary dictionary with sentiment statistics
    """
    # Create lookup for sentiment by submission_id
    sentiment_lookup = {s["submission_id"]: s for s in sentiment_analysis}

    # Count sentiments
    sentiment_counts = Counter(s["sentiment"] for s in sentiment_analysis)

    # Calculate average intensity
    intensities = [s["intensity"] for s in sentiment_analysis]
    avg_intensity = sum(intensities) / len(intensities) if intensities else 0.0

    # Determine overall sentiment
    if sentiment_counts["negative"] > sentiment_counts["positive"]:
        overall_sentiment = "negative"
    elif sentiment_counts["positive"] > sentiment_counts["negative"]:
        overall_sentiment = "positive"
    else:
        overall_sentiment = "neutral"

    # Sentiment by department
    sentiment_by_department = defaultdict(lambda: {"positive": 0, "neutral": 0, "negative": 0})
    for entry in feedback:
        submission_id = entry.get("submission_id")
        department = entry.get("job_area", "Unknown")
        sentiment = sentiment_lookup.get(submission_id, {}).get("sentiment", "neutral")
        sentiment_by_department[department][sentiment] += 1

    # Sentiment by category
    sentiment_by_category = defaultdict(lambda: {"positive": 0, "neutral": 0, "negative": 0})
    for entry in feedback:
        submission_id = entry.get("submission_id")
        category = entry.get("category", "Unknown")
        sentiment = sentiment_lookup.get(submission_id, {}).get("sentiment", "neutral")
        sentiment_by_category[category][sentiment] += 1

    return {
        "overall_sentiment": overall_sentiment,
        "positive_count": sentiment_counts.get("positive", 0),
        "neutral_count": sentiment_counts.get("neutral", 0),
        "negative_count": sentiment_counts.get("negative", 0),
        "average_intensity": round(avg_intensity, 2),
        "sentiment_by_department": {
            dept: dict(counts) for dept, counts in sentiment_by_department.items()
        },
        "sentiment_by_category": {
            cat: dict(counts) for cat, counts in sentiment_by_category.items()
        }
    }

